# Fast linear algebra libraries

Estimations using Deep Neural Networks involve large amount of operations that can be paralellized efficiently, such as matrix multiplications or applying non - linear function to every element in a matrix. Hence efficient libaries for such computations were developed. Examples of these libraries are `theano` and `tensorflow`.

This tutorial provides quick introduction into usage of `theano` library.

### Simple dataset

A cos function with noise added.

In [24]:
import numpy as np
import theano
import theano.tensor as T

from sklearn.model_selection import train_test_split

def rnd(*args):
    return np.random.randn(*args).astype(theano.config.floatX)

N = 400
X = rnd(N, 1)
y = X*0.8 + rnd(N, 1) * 0.05
y = np.cos(y*3.0) + 1.0

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

/home/user/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


### Define a computational graph here

In [25]:
# placeholders for inputs
Xv = T.matrix('X')
yv = T.matrix('y')

# variables of NN
n_neurons = 32
Wv = theano.shared(rnd(1, n_neurons), name="W")
bv = theano.shared(rnd(n_neurons), name="b")
sv = theano.shared(rnd(n_neurons, 1), name="s")
cv = theano.shared(rnd(1), name="c")

# computational graph of NN

# linear layer
Hv = T.dot(Xv, Wv) + bv

# Leaky Rectified Linear Unit activation
Hv = T.maximum(Hv, Hv*0.05)

# linear layer
Hv = T.dot(Hv, sv) + cv

# objective
cost = T.mean((yv - Hv) ** 2)

# automatic differentiation
V = [Wv, bv, sv, cv] # 
G = T.grad(cost, V)

# compile the procedure for estimations
estimate = theano.function(inputs=[Xv], outputs=[Hv])

# compile procedure for the gradient descent
gradstep = theano.function(inputs=[Xv, yv], outputs=[cost], updates=[
    [v, v - 0.01*g] for v, g in zip(V, G)
])

In [26]:
# call compiled function just like a normal python function
print(estimate(X_train[:3]))

[array([[ 0.40339421],
       [-1.29236308],
       [ 1.88511584]])]


In [27]:
theano.printing.pydotprint(estimate, outfile="g.png", var_with_name_simple=True) 

The output file is available at g.png


![Graph](g.png)

In [28]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import numpy as np
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from sklearn.metrics import r2_score
output_notebook()

# get outputs for the model on domain of dataset
Xm = np.linspace(-3.2, 3.2, 200)[:, np.newaxis]
ym = estimate(Xm)[0]

p = figure(title="Neural net in theano", plot_height=400, plot_width=700, y_range=(-5,5))

m = p.line(Xm[:,0], ym[:,0], color="blue", line_width=3, legend="NN outputs")
r = p.scatter(X_test[:,0], y_test[:,0], color="red", legend="Test data")

# will print here stats
out = widgets.Output()

def update(event):
    # update model on button click
    for i in range(100):
        gradstep(X_train, y_train)
    
    # evaluate the model
    yp_test = estimate(X_test)[0]
    score = r2_score(y_test, yp_test)
    
    # visualize the model
    yp = estimate(Xm)[0]
    m.data_source.data['y'] = yp[:, 0]
    push_notebook()
    
    with out:
        clear_output(wait=True)
        print("Model score: %s" % score)
        
show(p, notebook_handle=True)
step_b = widgets.Button(description="Iterate")
step_b.on_click(update)

display(widgets.VBox([
    widgets.HBox([step_b]), 
    out
]))

Loading BokehJS ...

VBox(children=(HBox(children=(Button(description='Iterate', style=ButtonStyle()),)), Output()))